<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras

In [37]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra_eng_sc_deeplearning.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:2]


In [38]:
print(pairs)

[['deep learning is a type of artificial intelligence derived from machine learning', "apprentissage profond est un type d'intelligence dérivé du apprentissage automatique"], ['A recurrent neural network is a class of artificial neural networks ', 'Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels']]


In [39]:
def clean_sentence(sentence):
    
    lower_case_sent = sentence.lower()
  
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [2]:
# from Book1 import words 
# print(words[0])
# for word in words:
#   z=word['English']+'\t'+word['French']
# #   raw_data3.append(z)
# text_file = open("sample.txt", "wt")
with open('fra_eng_sc_deeplearning.txt', 'w') as f:
  z="deep learning is a type of artificial intelligence derived from machine learning"+'\t'+"apprentissage profond est un type d'intelligence dérivé du apprentissage automatique"
  f.write("%s\n" % z)
  z="A recurrent neural network is a class of artificial neural networks "+'\t'+"Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels"
  f.write("%s\n" % z)
  # for word in words:
  #   z=word['English'].split('\/')[0]+'\t'+word['French'].split('\/')[0]
  #   f.write("%s\n" % z)

f.close()


In [40]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [42]:
english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]
french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]


fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))



french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 13
Maximum length english sentence: 12
french vocabulary is of 18 unique words
English vocabulary is of 17 unique words


Save french vocabulary



In [16]:
# Check language length

contant1=str(french_sentences)
file=open("french_sentences.txt","w+")
file.write(contant1)
file.close()


In [43]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,23, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,16, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)
print(max_french_len)
print(max_english_len)

13
12


In [18]:
print(max_english_len)

16


In [44]:
input_sequence = Input(shape=(23,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

In [25]:
encoder = LSTM(128, return_sequences=False)(embedding)
r_vec = RepeatVector(16)(encoder)
decoder = LSTM(128, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [26]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 23)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 23, 256)           7248128   
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 repeat_vector_1 (RepeatVect  (None, 16, 128)          0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 16, 128)           131584    
                                                                 
 time_distributed_1 (TimeDis  (None, 16, 15381)        1984149   
 tributed)                                                 

In [45]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=64, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 23ms/step - loss: 8.2819 - accuracy: 0.3438
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 5.3556 - accuracy: 0.3438
Epoch 3/100
1/1 [==============================] - 0s 17ms/step - loss: 3.7532 - accuracy: 0.3750
Epoch 4/100
1/1 [==============================] - 0s 18ms/step - loss: 3.3683 - accuracy: 0.2812
Epoch 5/100
1/1 [==============================] - 0s 18ms/step - loss: 2.8676 - accuracy: 0.2500
Epoch 6/100
1/1 [==============================] - 0s 19ms/step - loss: 2.4762 - accuracy: 0.3438
Epoch 7/100
1/1 [==============================] - 0s 17ms/step - loss: 2.1701 - accuracy: 0.4688
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 1.6929 - accuracy: 0.5938
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 1.2508 - accuracy: 0.6250
Epoch 10/100
1/1 [==============================] - 0s 18ms/step - loss: 0.8898 - accuracy: 0.6875
Epoch 11/100
1/1 [=

In [48]:
enc_dec_model.save('finalv_2_Model.h5') 

In [47]:

x=["apprentissage profond est un type d'intelligence dérivé du apprentissage automatique"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, 23, padding = "post")

# print(y)
z=np.reshape(z,(1,23,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)]).rstrip()

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(z)[0], eng_text_tokenizer))

The french sentence is: apprentissage profond est un type d'intelligence dérivé du apprentissage automatique
The predicted sentence is :
deep learning is a type of artificial intelligence derived from machine learning


In [ ]:
kldklsd